In [1]:
import sys
import os

IN_COLAB = False
DATA_PATH = "data/"

# if in colab, make colab setup
if 'google.colab' in sys.modules:
    
    IN_COLAB = True
    SSH_DIR = '/root/.ssh'
    ZIP_PATH = "/content/drive/MyDrive/embedded_ml_data/VOCdevkit.zip"
    FILENAME = "dev-notebook.ipynb"
    BRANCH = "michael"

    # Setup ssh-auth to github
    try:
        os.makedirs(SSH_DIR)
    except FileExistsError:
        # directory already exists
        pass

    !ssh-keyscan github.com >> /root/.ssh/known_hosts
    !echo 'PUBKEY' > /root/.ssh/id_rsa.pub
    !echo -e "PRIVKEY" > /root/.ssh/id_rsa
    !chmod 644 /root/.ssh/known_hosts
    !chmod 600 /root/.ssh/id_rsa
    !ssh -T git@github.com

    # Setup working environment
    !rm -rf models
    !rm -rf utils
    !git clone -b $BRANCH https://github.com/yannickfunk/EmbeddedMLLab tmp
    !rm tmp/$FILENAME
    !mv tmp/* .
    !rm -rf tmp
    !rm -rf sample_data
    %pip install -r requirements.txt

    # Setup data
    from google.colab import drive
    drive.mount('/content/drive')
    try:
      os.makedirs('./data')
    except FileExistsError:
      # directory already exists
      pass
    !cp  $ZIP_PATH data/
    %pushd data
    !unzip -qq VOCdevkit.zip
    %popd
    drive.flush_and_unmount()

    
try:
    os.makedirs('./checkpoints')
    os.makedirs('./checkpoints/results')
except FileExistsError:
    # directory already exists
    pass


Fix recursion error when using checkpointing

In [2]:
import sys

sys.setrecursionlimit(10000)

Prepare Tensorboard logger

In [3]:
# Setup logger
import pytorch_lightning as pl

LOGDIR='logs'
TENSORBOARD_DIR=LOGDIR+'/lightning_logs'

tensorboard = pl.loggers.TensorBoardLogger(save_dir=LOGDIR, default_hp_metric=True, log_graph=True)

try:
    os.makedirs('./'+LOGDIR+'/lightning_logs')
except FileExistsError:
    # directory already exists
    pass
 

In [4]:
%load_ext tensorboard
%tensorboard --logdir $TENSORBOARD_DIR

Prepare model

In [5]:
from models.tinyyolov2 import TinyYoloV2PersonOnly
import pytorch_lightning as pl
import torch

from utils.dataloader import VOCDataModule

from pytorch_lightning.callbacks.lr_monitor import LearningRateMonitor
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

import nni
from nni.compression.pytorch import LightningEvaluator


# Setting up callbacks
lr_monitor = LearningRateMonitor(logging_interval='step')
early_stopping = EarlyStopping(monitor="val_loss", mode='min', verbose=True)
checkpointing = ModelCheckpoint(
    save_top_k=3,
    monitor="val_loss",
    mode="min",
    verbose=True,
    auto_insert_metric_name=True,
    save_last=True
)

trainer = nni.trace(pl.Trainer)(
    max_epochs=50,
    auto_scale_batch_size='binsearch',
    accelerator="auto",
    devices=[0] if torch.cuda.is_available() else None,
    accumulate_grad_batches=1,
    logger=tensorboard,
    log_every_n_steps=1,
    fast_dev_run= True if not torch.cuda.is_available() else False,
    callbacks=[
    lr_monitor,
    early_stopping,
    checkpointing
    ]
)


model = TinyYoloV2PersonOnly(learning_rate=0.01)
model.load_pt_from_disk(DATA_PATH + "/voc_pretrained.pt", discard_layer_8=False)

data = nni.trace(VOCDataModule)(person_only=True)

evaluator = LightningEvaluator(trainer, data)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Tune and train model

In [6]:
evaluator.trainer.fit(model,data)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.


Using downloaded and verified file: data/VOCtrainval_11-May-2012.tar
Extracting data/VOCtrainval_11-May-2012.tar to data/



   | Name  | Type            | Params
-------------------------------------------
0  | loss  | YoloLoss        | 0     
1  | pad   | ReflectionPad2d | 0     
2  | conv1 | Conv2d          | 432   
3  | bn1   | BatchNorm2d     | 32    
4  | conv2 | Conv2d          | 4.6 K 
5  | bn2   | BatchNorm2d     | 64    
6  | conv3 | Conv2d          | 18.4 K
7  | bn3   | BatchNorm2d     | 128   
8  | conv4 | Conv2d          | 73.7 K
9  | bn4   | BatchNorm2d     | 256   
10 | conv5 | Conv2d          | 294 K 
11 | bn5   | BatchNorm2d     | 512   
12 | conv6 | Conv2d          | 1.2 M 
13 | bn6   | BatchNorm2d     | 1.0 K 
14 | conv7 | Conv2d          | 4.7 M 
15 | bn7   | BatchNorm2d     | 2.0 K 
16 | conv8 | Conv2d          | 9.4 M 
17 | bn8   | BatchNorm2d     | 2.0 K 
18 | conv9 | Conv2d          | 30.8 K
-------------------------------------------
9.5 M     Trainable params
6.3 M     Non-trainable params
15.8 M    Total params
63.058    Total estimated model params size (MB)
/opt/conda/envs/eml/l

Sanity Checking: 0it [00:00, ?it/s]

Using downloaded and verified file: data/VOCtrainval_11-May-2012.tar
Extracting data/VOCtrainval_11-May-2012.tar to data/


...

Using downloaded and verified file: data/VOCtrainval_11-May-2012.tar
Extracting data/VOCtrainval_11-May-2012.tar to data/


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 6.638
Epoch 0, global step 17: 'val_loss' reached 6.63835 (best 6.63835), saving model to 'logs/lightning_logs/version_21/checkpoints/epoch=0-step=17.ckpt' as top 3


PicklingError: Could not pickle object as excessively deep recursion required.

In [ ]:
from utils.ap import precision_recall_levels, ap, display_roc
from utils.yolo import nms, filter_boxes
from utils.dataloader import VOCDataLoaderPerson

import torch
import numpy as np
import tqdm


test_loader = VOCDataLoaderPerson(train=False, batch_size=1, data_path=DATA_PATH, n_limit=350)

test_precision = []
test_recall = []
model = model.load_from_checkpoint(checkpoint_path=checkpointing.best_model_path)

for inputs, targets in tqdm.tqdm(test_loader, total=350):
    with torch.no_grad():
        outputs = model(inputs, yolo=True)

    #The right threshold values can be adjusted for the target application
    outputs = filter_boxes(outputs, 0.0)
    outputs = nms(outputs, 0.5)
    outputs = torch.tensor(np.array(outputs))

    precision, recall = precision_recall_levels(targets[0], outputs[0])
    test_precision.append(precision)
    test_recall.append(recall)

avg_precision = ap(test_precision, test_recall)
print("average precision: ", avg_precision)
display_roc(test_precision, test_recall)

Using downloaded and verified file: data/VOCtrainval_11-May-2012.tar
Extracting data/VOCtrainval_11-May-2012.tar to data/


IsADirectoryError: [Errno 21] Is a directory: '/home/funnymonkey/eml'

# Pruning

In [ ]:
from tasks.pruning import auto_prune
from timeit import timeit

config_list = [{
    'op_types': ['Conv2d'],
    'total_sparsity': 0.5
},{
    'exclude': True,
    'op_names': [],
    'op_types': []
}]

best_person_only_model = model.load_from_checkpoint(checkpoint_path=checkpointing.best_model_path)



inference_time_unpruned = 

# Setting up callbacks
lr_monitor = LearningRateMonitor(logging_interval='step')

pruning_trainer = nni.trace(pl.Trainer)(
    auto_scale_batch_size='binsearch',
    accelerator="auto",
    devices=[0] if torch.cuda.is_available() else None,
    accumulate_grad_batches=1,
    logger=tensorboard,
    log_every_n_steps=1,
    fast_dev_run= True if not torch.cuda.is_available() else False,
    callbacks=[
    lr_monitor,  
    ]
)

pruned_model = auto_prune(model=best_person_only_model, trainer=pruning_trainer, datamodule=data, config_list=config_list)


SyntaxError: invalid syntax (2379398720.py, line 17)